In [1]:
import contacts as cts
import imessages as im

In [5]:
contact = cts.filter_contacts(firstname='ishani')[0]
contact

{'id': 8,
 'firstname': 'ishani 🐿️',
 'lastname': None,
 'phone': '5107503277',
 'email': None}

In [6]:
im_contact = im.get_contacts(f"id LIKE '%{contact["phone"]}%' OR id LIKE '%{contact["email"]}%'")[0]
im_contact

{'ROWID': 60,
 'id': '+15107503277',
 'country': 'US',
 'service': 'iMessage',
 'uncanonicalized_id': '5107503277',
 'person_centric_id': None}

In [7]:
if '@' in im_contact['id']:
    contact['email'] = im_contact['id']
else:
    contact['phone'] = im_contact['id']

contact

{'id': 8,
 'firstname': 'ishani 🐿️',
 'lastname': None,
 'phone': '+15107503277',
 'email': None}

---

In [2]:
import contacts as cts
import imessages as im

In [3]:
im.get_chat_participants('chat28488594168438315')

[{'chat_id': 'chat28488594168438315', 'sender_id': '+15107503277'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+14085041063'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+19254003943'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+16692687993'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+17145929889'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+14088962921'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+12068871643'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+14159919063'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+14086562467'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+15105090194'},
 {'chat_id': 'chat28488594168438315', 'sender_id': 'adrika.sen0409@gmail.com'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+14696166500'},
 {'chat_id': 'chat28488594168438315', 'sender_id': '+15303764084'}]

In [6]:
messages = im.get_messages(f"sender_id = '+15107503277'")
messages[-1]

{'dt': '2025-09-22 10:26:07',
 'guid': '63498FBC-60A9-4939-A0ED-8B78CC7341D3',
 'reply_to_guid': '6D06D8C1-1520-445F-A835-2AC7DD45DC95',
 'sender_id': '+15107503277',
 'chat_id': None,
 'chat_name': None,
 'is_from_me': 0,
 'content': 'where r u'}